In [9]:
# !  pip install emoji-translate spacy
# !pip install googletrans
# !pip install googletrans==4.0.0-rc1
! pip install emoji


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import datasets 
from datasets import load_dataset
from emoji_translate.emoji_translate import Translator
import googletrans 
emo = Translator(exact_match_only=False, randomize=True)
import re
from emoji import UNICODE_EMOJI


In [27]:
def is_emoji(s, language="en"):
    return s in UNICODE_EMOJI[language]

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

In [30]:
train_datset = pd.read_csv('/home/aatef/boda/ar_hate_speech_2022/data/OSACT2022-sharedTask-train.txt', sep='\t', header=None, index_col=0, names = ['sentence', 'OF_label','HS_label','Vulgar_label','Violence_label'])

In [31]:
train_dataset  = datasets.Dataset.from_pandas(train_datset)

In [32]:
train_dataset = train_dataset.select(range(10))

In [33]:
train_dataset[:10]['sentence']

['@USER ردينا ع التطنز 😏👊🏻',
 'وصارت فطاير البقالات غذاء صحي 👎🏻 URL',
 '@USER روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفوح يخقخق ويطالعهم بكل انحطاط ويصورهم الله يفشلكم بس😷',
 '@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔',
 '#اليوم_الاثنين<LF><LF>👏 يقولك :%90  من المسلمين <LF>عندهم خاله اسمها امل<LF>👏أو عمه امل <LF>👏او اخت امل<LF>👏او زوجه امل<LF>👏او بنت امل<LF>✊او حماة امل<LF>👊واللي ماعنده ولا امل<LF>بالعائلة يراجع نفسه لانه وضعه مو طبيعي/😳<LF>وأي قروب فيه "امل" هو من أسعد القروبات<LF>اذا عندكم وحده اسمها امل ارسلوها لها😽😻',
 'حمدلله ماحطها في فمي اساسا😷🤢 URL',
 '@USER هههه 😂🌚🤢',
 '#بايع_الكليجا اللي مايضحك من هالمقطع يبلكني هههههههه مطططههببللللل 😂😂😂👌👌👊👊😍😍😩😩😂😂 يمثلني ههههههه 😂😂👊 URL',
 'خلاص الله يزعجكم خلللاص 😷😷 #بايع_الكليجا',
 '@USER هيلق و جحلط غير كذا مافي 😷']

In [34]:
translator = googletrans.Translator()


In [35]:

s = 'ردينا ع التطنز smirk facepunch light skin tone'

en_words = re.finditer(r'[A-Za-z]+', s)
for match  in en_words:
    cur = str(match.group())
    s = re.sub(cur,'asd', s)
    print(s)
    print(match.group())
    print(match.string)
    

ردينا ع التطنز asd facepunch light skin tone
smirk
ردينا ع التطنز smirk facepunch light skin tone
ردينا ع التطنز asd asd light skin tone
facepunch
ردينا ع التطنز smirk facepunch light skin tone
ردينا ع التطنز asd asd asd skin tone
light
ردينا ع التطنز smirk facepunch light skin tone
ردينا ع التطنز asd asd asd asd tone
skin
ردينا ع التطنز smirk facepunch light skin tone
ردينا ع التطنز asd asd asd asd asd
tone
ردينا ع التطنز smirk facepunch light skin tone


In [36]:
def process(example):
    
    example['sentence'] = example['sentence'].replace('@USER', '') \
    .replace('URL', '') \
    .replace('<LF>','') \
    .replace('/', ' ') \
    .replace ('#', ' ')

    
    s = ''
    for i,c in  enumerate(example['sentence']):
        if is_emoji(c):
            s+= ' ' + c + ' '
        else:
            s+= c
       
    example['sentence'] = s 
    example['sentence'] = re.sub(' +', ' ', example['sentence'])
    
#     example['sentence'] =  add_space(example['sentence'])
    
    print( example['sentence'])
    
    example['sentence'] = emo.demojify (example['sentence'])

    en_words = re.finditer(r'[A-Za-z]+', example['sentence'])
    for match  in en_words:
        cur = str(match.group())
        example['sentence'] = re.sub(cur,translator.translate(cur, dest='ar').text, example['sentence'])  

    example['sentence'] = translator.translate( example['sentence'], dest='ar').text
    
    # print( example['sentence'])
    
    example['OF_label'] =  1 if example['OF_label'] == 'OFF' else 0
    example['Vulgar_label'] =  1 if example['Vulgar_label'] == 'VLG' else 0
    example['Violence_label'] =  1 if example['Violence_label'] == 'VIO' else 0
    example['HS_label'] =  0 if example['HS_label'] == 'NOT_HS' else int(example['HS_label'][2:])
    
    return example
    
    
    

In [37]:
processed_train_dataset = train_dataset.map(process,num_proc=1)

Parameter 'function'=<function process at 0x2aaba1da3820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

 ردينا ع التطنز 😏 👊 🏻 
ردينا ع التطنز سميرك وجه لكمة خفيفة جلد نغمة
وصارت فطاير البقالات غذاء صحي 👎 🏻 
وصارت فطاير البقالات غذاء صحي -1 خفيفة جلد نغمة
 روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفوح يخقخق ويطالعهم بكل انحطاط ويصورهم الله يفشلكم بس 😷 
روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفوح يخقخق ويطالعهم بكل انحطاط ويصورهم الله يفشلكم بس قناع
 مش باين حاجه خالص 😣 مش عارف بقى 😔 
مش باين حاجه خالص ثابر مش عارف بقى متأمل
#اليوم_الاثنين 👏 يقولك :%90 من المسلمين عندهم خاله اسمها امل 👏 أو عمه امل 👏 او اخت امل 👏 او زوجه امل 👏 او بنت امل ✊ او حماة امل 👊 واللي ماعنده ولا املبالعائلة يراجع نفسه لانه وضعه مو طبيعي/ 😳 وأي قروب فيه "امل" هو من أسعد القروباتاذا عندكم وحده اسمها امل ارسلوها لها 😽 😻 
#اليوم_الاثنين التصفيق يقولك :%90 من المسلمين عندهم خاله اسمها امل التصفيق أو عمه امل التصفيق او اخت امل التصفيق او زوجه امل التصفيق او بنت امل قبضة او حماة امل وجه لكمة واللي ماعنده ولا املبالعائلة يراجع نفسه لانه وضعه مو طبيعي/ ضجة وأي قروب فيه "امل" هو من أسعد القروباتاذا عندكم وحده اسمها امل 

In [38]:
processed_train_dataset[:5]

{'sentence': ['ردينا ع التطنز سميرك وجه لكمة خفيفة جلد نغمة',
  'وصارت فطاير البقالات غذاء صحي -1 خفيفة جلد نغمة',
  'روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفوح يخقخق ويطالعهم بكل انحطاط ويصورهم الله يفشلكم بس قناع',
  'مش باين حاجه خالص ثابر مش عارف بقى متأمل',
  '#اليوم_الاثنين التصفيق يقولك :%90 من المسلمين عندهم خاله اسمها امل التصفيق أو عمه امل التصفيق او اخت امل التصفيق او زوجه امل التصفيق او بنت امل قبضة او حماة امل وجه لكمة واللي ماعنده ولا املبالعائلة يراجع نفسه لانه وضعه مو طبيعي/ ضجة وأي قروب فيه "امل" هو من أسعد القروباتاذا عندكم وحده اسمها امل ارسلوها لها تقبيل قط قلب عيون قط'],
 'OF_label': [1, 0, 1, 0, 0],
 'HS_label': [0, 0, 0, 0, 0],
 'Vulgar_label': [0, 0, 0, 0, 0],
 'Violence_label': [0, 0, 0, 0, 0],
 '__index_level_0__': [1, 2, 3, 4, 5]}

## TODO 
 - remove english words before you convert emojis 
 - go over words and translate english ones 
 - convert numbers